## Building Toy model Of Glycolysis and Gluconeogenesis 

In [1]:
import os
import re
import warnings
from cobra.io.json import load_json_model as load_json_cobra_model
import escher
import numpy as np
import pandas as pd
import sympy as sym
from cobra import Model, Reaction, Metabolite, DictList
import cobra.test
from os.path import join
from six import iteritems
import cobra_dict as c_d
import csv
from minspan.minspan import minspan,nnz


In [2]:
maps_dir = os.path.abspath("maps")
data_dir = os.path.abspath("data")
dir = os.path.abspath("")
model_dir = os.path.abspath("models")
minspan_dir= os.path.abspath("minspans_csv")

#### Make empty COBRA model

In [3]:
# Initiate empty model
trial= cobra.Model()

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-04-30


In [4]:
g6p = Metabolite('G6P',compartment='c')
glu= Metabolite('Glucose',compartment='c')
f6p = Metabolite('F6P',compartment='c')
fdp = Metabolite('FDP',compartment='c')
pep = Metabolite('PEP',compartment='c')
pyr = Metabolite('PYR',compartment='c')


In [5]:
hex1 = Reaction('HEX1', lower_bound = 0)
g6pp = Reaction('G6PP', lower_bound = 0)
v1 = Reaction('v1', lower_bound=-1000)
pfk = Reaction('PFK', lower_bound = 0)
fbp = Reaction('FBP', lower_bound = 0)
v2 = Reaction('v2', lower_bound=-1000)
pyk = Reaction('PYK', lower_bound = 0)
pepck = Reaction('PEPCK', lower_bound = 0)
ex_glu = Reaction('EX__glu', lower_bound=-1000)
ex_pyr = Reaction('EX__pyr', lower_bound=-1000)


In [6]:
hex1.add_metabolites({glu : -1, g6p : 1})
g6pp.add_metabolites({glu : 1, g6p : -1})

v1.add_metabolites({g6p : -1,f6p : 1})

pfk.add_metabolites({f6p : -1, fdp : 1})
fbp.add_metabolites({f6p : 1, fdp : -1})

v2.add_metabolites({fdp : -1,pep : 1})

pyk.add_metabolites({pep : -1, pyr : 1})
pepck.add_metabolites({pep : 1, pyr : -1})

ex_glu.add_metabolites({glu : -1})
ex_pyr.add_metabolites({pyr : -1})


In [7]:
trial.add_reactions([hex1,v2,g6pp,pfk,fbp,v1,pyk,pepck,ex_glu,ex_pyr])

In [8]:
print(len(trial.reactions))
print(len(trial.metabolites))

10
6


In [9]:
from cobra.io.json import load_json_model

In [10]:
trial_json = os.path.join(model_dir,"gly_gluco_toy_model.json")
cobra.io.save_json_model(trial, trial_json)

In [11]:
model= load_json_model(trial_json)

In [12]:
# solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
# print(solved_fluxes)
# df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
# df = df/df.abs().max()
csv_dir = os.path.join(minspan_dir,"gly_toy_csv.csv")
# for col in range(len(df.columns)):
#     column=df.iloc[:,col]
#     vals = column.values
#     df[str(col)]=vals
# df.to_csv(csv_dir)
# df.to_csv(minspan_dir)
df = pd.read_csv(csv_dir, index_col =0)

# l = len(df.columns)
# h = int(l/2)
# df= df.iloc[:,h:l]
# minspan_df_norm.iloc[:,0] = minspan_df.iloc[:,2].values - minspan_df.iloc[:,0].values
df

,0,1,2,3,0.1,1.1,2.1,3.1
HEX1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
v2,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0
G6PP,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
PFK,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
FBP,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
v1,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0
PYK,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
PEPCK,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
EX__glu,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
EX__pyr,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0


In [13]:
l = len(df.columns)
h = int(l/2)
df= df.iloc[:,h:l]
df

,0.1,1.1,2.1,3.1
HEX1,0.0,0.0,0.0,1.0
v2,0.0,0.0,-1.0,0.0
G6PP,0.0,0.0,1.0,1.0
PFK,1.0,0.0,0.0,0.0
FBP,1.0,0.0,1.0,0.0
v1,0.0,0.0,-1.0,0.0
PYK,0.0,1.0,0.0,0.0
PEPCK,0.0,1.0,1.0,0.0
EX__glu,0.0,0.0,1.0,0.0
EX__pyr,0.0,0.0,-1.0,0.0


In [14]:
# df["4"]= df["0.1"] + df["1.1"] + df["3.1"]
df["5"]= -df["2.1"] + df["0.1"] + df["1.1"] + df["3.1"]

df= df.drop("1.1",axis=1).drop("0.1", axis=1).drop("3.1", axis=1)
df

C:\Users\sicil\AppData\Local\Temp\ipykernel_40868\2969990592.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["5"]= -df["2.1"] + df["0.1"] + df["1.1"] + df["3.1"]


,2.1,5
HEX1,0.0,1.0
v2,-1.0,1.0
G6PP,1.0,0.0
PFK,0.0,1.0
FBP,1.0,0.0
v1,-1.0,1.0
PYK,0.0,1.0
PEPCK,1.0,0.0
EX__glu,1.0,-1.0
EX__pyr,-1.0,1.0


In [15]:
import escher
from escher import Builder

In [16]:
initial_minspan_data = {
    reaction: flux
    for reaction, flux in df.iloc[:,0].iteritems()}

In [17]:
# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Gly_gluco_toy_1.json"),
    reaction_data=initial_minspan_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 0.0, 'v2': -1.0, 'G6PP': 1.0, 'PFK': 0.0, 'FBP': 1.0, 'v1': -1.0, 'PYK': 0.0, '…

In [18]:
trial.remove_reactions([v1,v2, pepck])

In [19]:
dhap= Metabolite('dhap',compartment='c')
g3p = Metabolite('g3p',compartment='c')
_13_dpg = Metabolite('13dpg',compartment='c')
_3_dpg = Metabolite('3dpg',compartment='c')
_2_dpg = Metabolite('2dpg',compartment='c')
oaa = Metabolite('oaa',compartment='c')

In [20]:

pgi = Reaction('PGI', lower_bound=-1000)
fba = Reaction('FBA', lower_bound=-1000)
gapd = Reaction('GAPD', lower_bound=-1000)
pgk = Reaction('PGK', lower_bound=-1000)
pgm = Reaction('PGM', lower_bound = -1000)
eno = Reaction('ENO', lower_bound = -1000)
pepck = Reaction('PEPCK', lower_bound = 0)
pc = Reaction('PC', lower_bound = 0)


In [21]:
pgi.add_metabolites({g6p : -1, f6p : 1})

fba.add_metabolites({fdp : -1, g3p : 1})

gapd.add_metabolites({g3p : -1,_13_dpg : 1})

pgk.add_metabolites({_13_dpg : -1, _3_dpg : 1})

pgm.add_metabolites({_3_dpg : -1, _2_dpg : 1})

eno.add_metabolites({_2_dpg : -1,pep : 1})

pepck.add_metabolites({pep : 1, oaa : -1})

pc.add_metabolites({oaa : 1, pyr : -1})

In [22]:
trial.add_reactions([pgi, fba, gapd, pgk, pgm, eno, pepck, pc])

In [23]:
print(len(trial.reactions))
print(len(trial.metabolites))

15
11


In [24]:
from cobra.io.json import load_json_model

In [25]:
trial_json = os.path.join(model_dir,"gly_gluco_toy_model_2.json")
cobra.io.save_json_model(trial, trial_json)

In [26]:
model= load_json_model(trial_json)

In [27]:
print(model.reactions.FBA.check_mass_balance())

{}


In [28]:
solved_fluxes = minspan(model, cores=3, verbose=False, timelimit=60)
print(solved_fluxes)
df = pd.DataFrame(solved_fluxes.copy(), index=[i.id for i in model.reactions])
df = df/df.abs().max()
csv_dir = os.path.join(minspan_dir,"gly_toy_2_csv.csv")
for col in range(len(df.columns)):
    column=df.iloc[:,col]
    vals = column.values
    df[str(col)]=vals
df.to_csv(csv_dir)

df = pd.read_csv(csv_dir, index_col =0)

# l = len(df.columns)
# h = int(l/2)
# df= df.iloc[:,h:l]
# minspan_df_norm.iloc[:,0] = minspan_df.iloc[:,2].values - minspan_df.iloc[:,0].values
df

Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpkkxvqywd.lp
Reading time = 0.00 seconds
: 11 rows, 30 columns, 56 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpo_fbhmtn.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpu7k_qxma.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp3pv_uauc.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp_6b9ktic.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmpmcgu07ur.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 nonzeros
Read LP format model from file C:\Users\sicil\AppData\Local\Temp\tmp3lz3dzwy.lp
Reading time = 0.00 seconds
: 41 rows, 45 columns, 139 no

,0,1,2,3,0.1,1.1,2.1,3.1
HEX1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
G6PP,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
PFK,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
FBP,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
PYK,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0
EX__glu,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0
EX__pyr,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
PGI,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
FBA,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
GAPD,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [29]:
l = len(df.columns)
h = int(l/2)
df= df.iloc[:,h:l]
df

,0.1,1.1,2.1,3.1
HEX1,0.0,1.0,0.0,1.0
G6PP,0.0,0.0,0.0,1.0
PFK,0.0,1.0,1.0,0.0
FBP,0.0,0.0,1.0,0.0
PYK,1.0,1.0,0.0,0.0
EX__glu,0.0,-1.0,0.0,0.0
EX__pyr,0.0,1.0,0.0,0.0
PGI,0.0,1.0,0.0,0.0
FBA,0.0,1.0,0.0,0.0
GAPD,0.0,1.0,0.0,0.0


In [30]:
import escher
from escher import Builder

In [43]:
initial_minspan_data = {
    reaction: flux
    for reaction, flux in df.iloc[:,3].iteritems()}

In [44]:
# New instance to prevent modifications to the existing maps
escher_builder = Builder(
    model=trial,
    map_json=os.path.join(
        maps_dir,"Gly_gluco_toy_2.json"),
    reaction_data=initial_minspan_data)

# Display map in notebook
escher_builder

Builder(reaction_data={'HEX1': 1.0, 'G6PP': 1.0, 'PFK': 0.0, 'FBP': 0.0, 'PYK': 0.0, 'EX__glu': 0.0, 'EX__pyr'…